# Predict BBT change using Kalman Filter
**Author:** Thuy Trinh

**Achievement:**

# Introduction
Aims:
1. to explain daily  fluctuations in BBT
2. to derive a predictive  distribution of menstrual cycle length that is dependent on the **current phrase** state

## Import packages

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as m
from numpy.linalg import inv
%matplotlib inline

# Data

In [203]:
data = pd.read_csv("bbtdata_forTask7.csv", index_col = "day")
data.head()

,bbt_subject1,bbt_subject2,bbt_subject3,bbt_subject4,bbt_subject5
day,,,,,
0,35.95,36.19,36.41,36.56,36.65
1,35.98,36.23,36.04,36.32,36.61
2,36.10,36.20,36.01,36.21,37.00
3,36.07,36.14,36.01,36.18,36.90
4,36.00,36.37,36.03,36.16,36.65


In [204]:
df1 = list(data['bbt_subject1'])
# df1

## Code

In [208]:
mea = list(data['bbt_subject1'])
err_mea = 4
err_est = 2
init_est = 37

est = init_est
est_total = [est]
err_est_total = [err_est]
KG_total = []
for value in mea:
    KG = err_est/(err_est + err_mea)
    est = est + KG*(value-est)
    err_est = (1-KG)*err_est
    
    KG_total.append(KG)
    est_total.append(est)
    err_est_total.append(err_est)

mea.insert(0, "na")
KG_total.insert(0,"na")
res = {"mea": mea,
      "est": est_total,
      "err_est": err_est_total,
      "KG": KG_total}
res_df = pd.DataFrame(res)
res_df.tail()

,mea,est,err_est,KG
250,36.12,36.250437,0.015873,0.00396825
251,36.1,36.249842,0.015810,0.00395257
252,36.22,36.249724,0.015748,0.00393701
253,36.16,36.249373,0.015686,0.00392157
254,36.2,36.249180,0.015625,0.00390625


In [207]:
# data
y_mea = list(data['bbt_subject1'])

# parameters
c = 0.5
b = 43
B = np.array([b, 0])
sigma_y = 0.5
a = 0.2
f = 1
delta_t = 1
sigma_x = 0.5
## compute A
radian = m.degrees(2 * m.pi * f * delta_t)
A = np.array([[a * (m.cos(radian) - m.sin(radian)), 0],
              [0, a * (m.sin(radian) - m.cos(radian))]])
I = np.identity(2)

# initialise values
cov_err_y_est = np.array([[sigma_y**2, 0],
                          [0, sigma_y**2]])
cov_err_x_est = np.array([[sigma_x**2, 0],
                          [0, sigma_x**2]])
init_x_est = np.array([22.5, 22.5])

x_est = init_x_est
x_est_total = []

# tobe compute from x
KG_total = []
y_est_total = []
err_y_est_total = []
cov_err_y_est_total = []

for value in y_mea:
    # compute x_est and y_est from x_est_init
    x_est = A.dot(x_est)
    y_est = B.dot(x_est) + c
    
    # compute covariance of error from x_est
    cov_err_x_est = A.dot(cov_err_x_est).dot(A.T) + sigma_x*I
#     print("cov_err_x_est", cov_err_x_est)
    # compute covariance of error from y_est
    cov_err_y_est = B.dot(cov_err_x_est).dot(B.T) + sigma_y
#     print("cov_err_y_est", cov_err_y_est)
    
    # compute Kalman gain
    err_est = (value - y_est)
    KG = cov_err_x_est.dot(B.T).dot(1/cov_err_y_est)
#     KG = err_est/(err_est + cov_err_y_est)
    
    # update x_est and error
    x_est = x_est + KG*err_est
    cov_err_x_est = cov_err_x_est - KG.dot(cov_err_y_est).dot(KG.T)
    
    KG_total.append(KG)
    x_est_total.append(x_est)
    err_y_est_total.append(err_est)
    y_est_total.append(y_est)
    cov_err_y_est_total.append(cov_err_y_est)
#     err_y_est_total.append(err_est)

# y_mea.insert(0, "na")
# KG_total.insert(0,"na")
res = {"mea": y_mea,
      "y_est": y_est_total,
      "y_err_est": err_est,
       "x_est": x_est_total,
      "KG": KG_total}
# print(res)
res_df = pd.DataFrame(res)
res_df.tail()

,mea,y_est,y_err_est,x_est,KG
249,36.12,-8.391053,44.556289,"[0.8278124586604518, 0.08043345941633467]","[0.023243241023407992, 0.0013559298057071974]"
250,36.10,-8.346350,44.556289,"[0.8273481558874957, 0.08025556431007769]","[0.023243241023407992, 0.0013559298057071974]"
251,36.22,-8.341389,44.556289,"[0.8301374071938368, 0.0803673373897505]","[0.023243241023407992, 0.0013559298057071974]"
252,36.16,-8.371196,44.556289,"[0.8287424379697591, 0.08035417594440132]","[0.023243241023407992, 0.0013559298057071974]"
253,36.20,-8.356289,44.556289,"[0.8296723550381941, 0.08038492908549445]","[0.023243241023407992, 0.0013559298057071974]"


In [200]:
# compute loglikelihood
T = len(y_est_total)
component_1 = -(T/2)*np.log(2*m.pi)
component_2 = (np.log(err_y_est_total))
diff_y = np.array([np.array(y_mea)-np.array(y_est_total)])
print(diff_y.shape)
inverse_F = 1./np.array([cov_err_y_est_total])
component_3 = (diff_y.T).dot(inverse_F).dot(diff_y.T)
logL = component_1 - 0.5*sum(i for i in component_2) - 0.5*sum(i for i in component_3)
logL[0]

(1, 254)


-72975.45560667577

In [122]:
import numpy as np
from numpy.linalg import inv

x_observations = np.array([4000, 4260, 4550, 4860, 5110])

z = np.c_[x_observations, v_observations]

# Initial Conditions
a = 2  # Acceleration
v = 280
t = 1  # Difference in time

# Process / Estimation Errors
error_est_x = 20

# Observation Errors
error_obs_x = 25  # Uncertainty in the measurement

def prediction2d(x, t, a):
    A = np.array([[1, t],
                  [0, 1]])
    X = np.array([x])
    X_prime = A.dot(X) 
    return X_prime


def covariance2d(sigma1, sigma2):
    cov1_2 = sigma1 * sigma2
    cov2_1 = sigma2 * sigma1
    cov_matrix = np.array([[sigma1 ** 2, cov1_2],
                           [cov2_1, sigma2 ** 2]])
    return np.diag(np.diag(cov_matrix))


# Initial Estimation Covariance Matrix
P = covariance2d(error_est_x, error_est_v)
A = np.array([[1, t],
              [0, 1]])

# Initial State Matrix
X = np.array([[z[0][0]],
              [v]])
n = len(z[0])

for data in z[1:]:
    X = prediction2d(X[0][0], X[1][0], t, a)
    # To simplify the problem, professor
    # set off-diagonal terms to 0.
    P = np.diag(np.diag(A.dot(P).dot(A.T)))

    # Calculating the Kalman Gain
    H = np.identity(n)
    R = covariance2d(error_obs_x, error_obs_v)
    S = H.dot(P).dot(H.T) + R
    K = P.dot(H).dot(inv(S))

    # Reshape the new data into the measurement space.
    Y = H.dot(data).reshape(n, -1)

    # Update the State Matrix
    # Combination of the predicted state, measured values, covariance matrix and Kalman Gain
    X = X + K.dot(Y - H.dot(X))

    # Update Process Covariance Matrix
    P = (np.identity(len(K)) - K.dot(H)).dot(P)

print("Kalman Filter State Matrix:\n", X)

TypeError: prediction2d() takes 3 positional arguments but 4 were given

### apply the code to our problem

In [114]:
import numpy as np
from numpy.linalg import inv

x_observations = np.array([4000, 4260, 4550, 4860, 5110])
v_observations = np.array([280, 282, 285, 286, 290])

z = np.c_[x_observations, v_observations]

# Initial Conditions
a = 2  # Acceleration
v = 280
t = 1  # Difference in time

# Process / Estimation Errors
error_est_x = 20
error_est_v = 5

# Observation Errors
error_obs_x = 25  # Uncertainty in the measurement
error_obs_v = 6

def prediction2d(x, v, t, a):
    A = np.array([[1, t],
                  [0, 1]])
    X = np.array([[x],
                  [v]])
    B = np.array([[0.5 * t ** 2],
                  [t]])
    X_prime = A.dot(X) + B.dot(a)
    return X_prime


def covariance2d(sigma1, sigma2):
    cov1_2 = sigma1 * sigma2
    cov2_1 = sigma2 * sigma1
    cov_matrix = np.array([[sigma1 ** 2, cov1_2],
                           [cov2_1, sigma2 ** 2]])
    return np.diag(np.diag(cov_matrix))


# Initial Estimation Covariance Matrix
P = covariance2d(error_est_x, error_est_v)
A = np.array([[1, t],
              [0, 1]])

# Initial State Matrix
X = np.array([[z[0][0]],
              [v]])
n = len(z[0])

for data in z[1:]:
    X = prediction2d(X[0][0], X[1][0], t, a)
    # To simplify the problem, professor
    # set off-diagonal terms to 0.
    P = np.diag(np.diag(A.dot(P).dot(A.T)))

    # Calculating the Kalman Gain
    H = np.identity(n)
    R = covariance2d(error_obs_x, error_obs_v)
    S = H.dot(P).dot(H.T) + R
    K = P.dot(H).dot(inv(S))

    # Reshape the new data into the measurement space.
    Y = H.dot(data).reshape(n, -1)

    # Update the State Matrix
    # Combination of the predicted state, measured values, covariance matrix and Kalman Gain
    X = X + K.dot(Y - H.dot(X))

    # Update Process Covariance Matrix
    P = (np.identity(len(K)) - K.dot(H)).dot(P)

print("Kalman Filter State Matrix:\n", X)

Kalman Filter State Matrix:
 [[5127.05898493]
 [ 288.55147059]]


### apply the code from this [source]("https://medium.com/@jaems33/understanding-kalman-filters-with-python-2310e87b8f48") 

In [65]:
y_observations = np.array([36.5, 36, 37, 37.5, 36.5, 37, 37.5])

# Initial Conditions
a = 1 
f = 1
delta_t = 0.01
b = 1
B = np.array([b, 0])
c = 1

X = np.array([36.29, 30])
I = np.identity(2)

# Process / Estimation Errors
sigma_x = 0.01
E = np.array([[sigma_x**2, 0],
              [0, sigma_x**2]])

# Observation Errors
sigma_y = 0.01  # Uncertainty in the measurement

def x_estimate(x_pre, A):
    x_cur = A.dot(x_pre)
    return (x_cur)

def covariance_x_estimate(covariance_x_pre, A, sigma_x):
    I = np.identity(2)
    covariance_x_cur = A.dot(covariance_x_pre).dot(A.T) + sigma_x*I
    return covariance_x_cur

def y_estimate(x_estimate, b, c):
    y_est = b.dot(x_estimate) + c
    return y_est

def covariance_y_estimate(covariance_x_est, b, simga_y):
    covariance_y_est = b.dot(covariance_x_est).dot(b.T) + sigma_y
    return covariance_y_est

def kalman_gain(covariance_x_estimate, b, covariance_y_estimate):
    K = (covariance_x_estimate.dot(b.T))*covariance_y_estimate**(-1)
    return K

def x_update(x_estimate, K, y_estimate, y_observe):
    x_update = x_estimate + K*(y_observe-y_estimate)
    return x_update

def covariance_x_update(cov_x_pred, K, cov_y_pred):
    covariance_x_update = cov_x_pred + K*cov_y_pred*K
    return covariance_x_update

# # Initial State Matrix
# X = np.array([(y_observations[0]-c)/b[0], 0])
# I = np.identity(2)
# print("\n X_initial in degree=:\n", X)


# radian = m.degrees(2 * m.pi * f * delta_t)

# A = np.array([[a * (m.cos(radian) - m.sin(radian)), 0],
#               [0, a * (m.sin(radian) + m.cos(radian))]])

# x_pred = x_estimate(X, A)
# print("\n X_predict:\n", x_pred)

# cov_x_pred = covariance_x_estimate(C, A, sigma_x)
# print("\n covariance_x_predict:\n", cov_x_predict)

# y_est = y_estimate(x_pred, b, c)
# print("\n y_predict:\n", y_est)

# cov_y_pred = covariance_y_estimate(cov_x_pred, b, sigma_y)
# print("\n covariance_y_predict:\n", cov_y_pred)

# K = kalman_gain(cov_x_pred, b, cov_y_pred)
# print("\n kalman_gain:\n", K)

# y_observe = y_observations[1]
# x_upd = x_update(x_pred, K, y_est, y_observe)
# print("\n x_update:\n", x_upd)

# covariance_x_upd = covariance_x_update(cov_x_pred, K, cov_y_pred)
# print("\n covariance_x_update:\n", covariance_x_upd)

radian = m.degrees(2 * m.pi * f * delta_t)

A = np.array([[a * (m.cos(radian) - m.sin(radian)), 0],
              [0, a * (m.sin(radian) + m.cos(radian))]])

for data in y_observations:
    print("\n X_pre=\n", X)
    
    ## estimate X and cov_x
#     X = x_estimate(X, A)
    X = A.dot(X)
    print("\n X_predict:\n", X)
    E = covariance_x_estimate(E, A, sigma_x)
#     print("\n covariance_x_predict:\n", C)

    ## estimate y and cov_y
    y = y_estimate(X, B, c)
    print("\n y_predict:\n", y)
    F = covariance_y_estimate(E, B, sigma_y)
#     print("\n covariance_y_predict:\n", F)

    # Calculate Kalman Gain
    K = kalman_gain(E, B, F)
#     print("\n kalman_gain:\n", K)
    
    # update X and cov_x
    y_observe = data
    X = x_update(X, K, y, y_observe)
    print("\n x_update:\n", X)
    E = covariance_x_update(E, K, F)
#     print("\n covariance_x_update:\n", C)


 X_pre=
 [36.29 30.  ]

 X_predict:
 [-16.48429604 -40.17836579]

 y_predict:
 -15.484296041595996

 x_update:
 [  9.53463942 -40.17836579]

 X_pre=
 [  9.53463942 -40.17836579]

 X_predict:
 [-4.33099528 53.81003592]

 y_predict:
 -3.3309952841525696

 x_update:
 [17.97538853 59.00454109]

 X_pre=
 [17.97538853 59.00454109]

 X_predict:
 [ -8.16510405 -79.0235345 ]

 y_predict:
 -7.165104048459611

 x_update:
 [ 17.77759438 -67.40341034]

 X_pre=
 [ 17.77759438 -67.40341034]

 X_predict:
 [-8.07525844 90.27196253]

 y_predict:
 -7.075258436541242

 x_update:
 [ 18.42624685 106.47833635]

 X_pre=
 [ 18.42624685 106.47833635]

 X_predict:
 [  -8.36990102 -142.60418488]

 y_predict:
 -7.369901022134876

 x_update:
 [  17.82774902 -123.59175546]

 X_pre=
 [  17.82774902 -123.59175546]

 X_predict:
 [ -8.09804058 165.52382532]

 y_predict:
 -7.098040579166456

 x_update:
 [ 18.27945308 186.66495193]

 X_pre=
 [ 18.27945308 186.66495193]

 X_predict:
 [  -8.30322172 -249.99642395]

 y_pred

# Result

Expected Result:
<img src="pictures/sample_result.png">